In [16]:
import os.path
import sys
sys.path.append('/Users/cmagri1/OneDrive - Johns Hopkins/git')
from torch import nn
import torch
from torchvision.models.alexnet import alexnet
from tqdm import tqdm

from EncodingModel_cm296 import utils as emutils
from EncodingModel_cm296 import feature_extractor as emfe

import pandas as pd
import numpy as np
import csv

from utilsCM import Subject
from utilsCM import cv_regression #Predicting fMRI reponses with features (Alexnet activations) 
#cv_regression computes cross-validated ridge regression. cross-validation groupings were pre-set based
#on fMRI design. 9-categories out, 9 folds. r is averaged over folds, weights are computed over all data


In [17]:
# specify variables
resolutionval = 227;
layer = {'conv_1','conv_2','conv_3','conv_4','conv_5'};
# print(model)


#sepcify directory where images (for which we have fMRI data) are
PathToImgs = 'data-object2vec/stimuli'
# conditions = emutils.listdir('data-object2vec/stimuli')
# keyword = 'object2vec'

In [18]:
# extract fMRI data
ROI = 'LOC'
Sub = 1
subject = Subject(Sub,[ROI])

In [19]:
#Load word2sense
#already preprocessed in MATLAB so that Wrd2Sns and THINGs overlap --> we have IMAGES, LABELS and SENSES
pathtofile = '../Code-001/'
Wrd2Sense = pd.read_csv(pathtofile + "ThingsWrd2Sns.txt", sep=",")
ImgInfo = pd.read_csv(pathtofile + "KeptTHINGSInfo.txt", sep=",")
pathtoTHINGS = '/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense//THINGSdataset/Main/images'

In [20]:
def object_by_feature(PathToImgs,keyword,ilayer,resolutionval):
    datapath = 'data/'
    if os.path.isfile(datapath+ keyword + '_' + ilayer + ".csv"):
        print('loading file with '+ keyword+ ' stimuli features for ' + ilayer)
        features = pd.read_csv(datapath+ keyword + "_" + ilayer + ".csv"  , sep=",", header=None, index_col=0)
        condition_features = {}
        for index, row in features.iterrows():
            condition_features[index] = row 
    else:
        print('Computing '+ keyword+ ' stimuli features for ' + ilayer)
        model = emfe.AlexNet(ilayer); 
        
        condition_features = emutils.compute_features(model, PathToImgs,resolutionval)
        features = pd.DataFrame(condition_features).transpose().set_index(0)  
        features.to_csv(datapath + keyword+ "_" + ilayer + ".csv", index = True, header=False)
#     layer_by_features[ilayer] = features; #create multistructure with all layers
    return condition_features

In [23]:
#Compute features of CNN
for ilayer in layer:
    #---------Object2Vect-----------------
    keyword = 'object2vec'
    object2vec_features = object_by_feature(PathToImgs,keyword,ilayer,resolutionval)
    weights, r = cv_regression(object2vec_features, subject, l2=0)
    
    
    #---------THINGS-----------------
    keyword = 'things'
    things_features = object_by_feature(pathtoTHINGS,keyword,ilayer,resolutionval)
    things_features_df = pd.DataFrame(things_features).transpose()
    
    #---Compute Predicted ROI response to THINGS dataset
    print("Computing ROI prediction for " + ilayer)
    ROIpred = {}
    ROIpred = np.dot(things_features_df.iloc[:,:].to_numpy(),weights.transpose())
    np.save(os.getcwd() + '/data/ROIpred_Sub' + str(Sub) + '_' + ROI + "_" + ilayer, ROIpred)

print("Done!")

Computing object2vec stimuli features for conv_3


100%|██████████| 81/81 [00:19<00:00,  4.25it/s]


loading file with things stimuli features for conv_3
Computing ROI prediction for conv_3
Computing object2vec stimuli features for conv_2


100%|██████████| 81/81 [00:16<00:00,  4.77it/s]


loading file with things stimuli features for conv_2
Computing ROI prediction for conv_2
loading file with object2vec stimuli features for conv_1
loading file with things stimuli features for conv_1
Computing ROI prediction for conv_1
loading file with object2vec stimuli features for conv_5
loading file with things stimuli features for conv_5
Computing ROI prediction for conv_5
Computing object2vec stimuli features for conv_4


100%|██████████| 81/81 [00:20<00:00,  3.93it/s]


loading file with things stimuli features for conv_4
Computing ROI prediction for conv_4
Done!
